In [42]:
import autogen
from aitools_autogen.config import llm_config_llama2


In [43]:
config_list_openai = [
    {
        'base_url': 'http://aitools.cs.vt.edu:7860/openai/v1',
        'api_key': 'aitools',
        'model': 'gpt-4-turbo-preview',
    }
]

llm_config_openai = {
    "timeout": 300,
    "seed": 42,
    "config_list": config_list_openai,
    "temperature": 0.1,
    "allow_format_str_template": True
}

# Define an assistant coder using the LLM
coder = autogen.AssistantAgent(
    name="Coder",
    llm_config=llm_config_openai,
    system_message="""
    You are a professional Python coder, known for your insightful and engaging solutions with explanations to coding problems.
    You produce Python code in response to coding problems.  You do not comment on feedback, you respond ONLY with code.
    You should improve the quality of the code based on the feedback from the user.
    When the user indicates your code is acceptable, send a TERMINATE message.
    """,
)


In [44]:
# Define the user proxy, the driver of the conversation who has ability to execute code.
user_proxy = autogen.UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    max_consecutive_auto_reply=6, # stop infinite chat loops
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "tasks",
        "use_docker": False,
    },
    llm_config=llm_config_llama2
)

[autogen.oai.client: 03-31 11:01:00] {123} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [45]:
# Example Task
task = f"""
Write a function in Python to sort an array of integers using quicksort.
"""


In [46]:
res = user_proxy.initiate_chat(recipient=coder, message=task, max_turns=2, summary_method="last_msg")
print(res)
print("---")
print(user_proxy.last_message(coder))

No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


User (to Coder):


Write a function in Python to sort an array of integers using quicksort.


--------------------------------------------------------------------------------


No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


Coder (to User):

```python
def quicksort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]
    return quicksort(left) + middle + quicksort(right)

# Example usage
arr = [3, 6, 8, 10, 1, 2, 1]
print(quicksort(arr))
```

--------------------------------------------------------------------------------


No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.



>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...


No default IOStream has been set, defaulting to IOConsole.


User (to Coder):

exitcode: 0 (execution succeeded)
Code output: 
[1, 1, 2, 3, 6, 8, 10]


--------------------------------------------------------------------------------


No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


Coder (to User):

TERMINATE

--------------------------------------------------------------------------------
ChatResult(chat_id=None, chat_history=[{'content': '\nWrite a function in Python to sort an array of integers using quicksort.\n', 'role': 'assistant'}, {'content': '```python\ndef quicksort(arr):\n    if len(arr) <= 1:\n        return arr\n    pivot = arr[len(arr) // 2]\n    left = [x for x in arr if x < pivot]\n    middle = [x for x in arr if x == pivot]\n    right = [x for x in arr if x > pivot]\n    return quicksort(left) + middle + quicksort(right)\n\n# Example usage\narr = [3, 6, 8, 10, 1, 2, 1]\nprint(quicksort(arr))\n```', 'role': 'user'}, {'content': 'exitcode: 0 (execution succeeded)\nCode output: \n[1, 1, 2, 3, 6, 8, 10]\n', 'role': 'assistant'}, {'content': 'TERMINATE', 'role': 'user'}], summary='', cost=({'total_cost': 0.007509999999999999, 'gpt-4-0125-preview': {'cost': 0.007509999999999999, 'prompt_tokens': 379, 'completion_tokens': 124, 'total_tokens': 503}}, {'